In [4]:
#pip install pyathena

In [5]:
#Libraries
from pyathena import connect
import pandas as pd
import datetime as dt
import numpy as np

In [7]:
from pyathena import connect
import pandas as pd
conn = connect(s3_staging_dir='s3://viamericas-datalake-dev-us-east-1-283731589572-athena/queryresults/',
               region_name='us-east-1')
df = pd.read_sql(''' 
SELECT
    CAST(RTRIM(p.NAME_MAIN_BRANCH) AS VARCHAR(60)) AS PAYER,
    RTRIM(co.NAME_COUNTRY) AS COUNTRY,
    CAST(DATE_FORMAT(a.DATE_RECEIVER, '%Y-%m-%d') AS DATE) AS DATE,
    a.ID_SENDER,
    a.ID_BRANCH,
    s.UUID,
    SUM(CASE WHEN (a.ID_FLAG_RECEIVER = 'A' OR a.ID_FLAG_RECEIVER = 'C') THEN 0 ELSE a.NET_AMOUNT_RECEIVER END) AS AMOUNT
FROM
    AwsDataCatalog.viamericas.RECEIVER a
    INNER JOIN AwsDataCatalog.viamericas.GROUP_BRANCH p ON p.ID_MAIN_BRANCH = CASE WHEN a.ID_MAIN_BRANCH_EXPIRED IS NULL THEN RTRIM(a.ID_MAIN_BRANCH_SENT) ELSE RTRIM(a.ID_MAIN_BRANCH_EXPIRED) END
    INNER JOIN AwsDataCatalog.viamericas.COUNTRY co ON a.ID_COUNTRY_RECEIVER = co.ID_COUNTRY
    LEFT JOIN AwsDataCatalog.viamericas.BRANCH b ON a.ID_BRANCH = b.ID_BRANCH
    LEFT JOIN AwsDataCatalog.viamericas.SENDER s ON a.ID_SENDER = s.id_sender AND a.ID_BRANCH = s.ID_BRANCH -- Union con la tabla SENDER
WHERE
    NOT (a.ID_MAIN_BRANCH_SENT LIKE 'M%')
    AND NOT (a.ID_BRANCH LIKE 'T%')
    AND SUBSTRING(b.id_branch, 1, 1) IN (SELECT SUBSTRING(PREFIX, 1, 1) FROM AwsDataCatalog.viamericas.BRANCH_PREFIX P WHERE TRIM(b.ID_COUNTRY) = TRIM(P.ID_COUNTRY))
    AND a.DATE_RECEIVER >= TIMESTAMP '2022-06-01 00:00:00.000'
    AND a.DATE_RECEIVER < TIMESTAMP '2022-06-30 00:00:00.000'
    AND b.ID_LOCATION IS NOT NULL
    AND b.ID_LOCATION NOT IN ('MD0010', 'MD0952', 'AK0003', 'CA3897', 'NY1130', 'MD0696', 'FL1933', 'AK0004', 'CA4046', 'NY1221', 'MD0623', 'MD1003', 'MD1018', 'AK0008', 'CA4291', 'NY1346', 'CA4350', 'NY1381','OK0236','FL2287','CA4391','AK0009', 'NY1397','CA4392','AK0010','FL2288','NY1399','AK0012','CA4396','NY1402','FL2289','AK0013', 'CA4418','NY1410','FL2301','AK0014', 'CA4428', 'NY1413')
GROUP BY
    RTRIM(p.NAME_MAIN_BRANCH),
    RTRIM(co.NAME_COUNTRY),
    CAST(DATE_FORMAT(a.DATE_RECEIVER, '%Y-%m-%d') AS DATE),
    a.ID_SENDER,
    a.ID_BRANCH,
    s.UUID
''', conn)

/tmp/ipykernel_35/1259278767.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql('''


In [9]:
df.UUID.nunique()

835811

In [14]:
df = df[df['AMOUNT'] != 0] # Limpio los 0

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1511181 entries, 0 to 1566739
Data columns (total 7 columns):
 #   Column     Non-Null Count    Dtype  
---  ------     --------------    -----  
 0   PAYER      1511181 non-null  object 
 1   COUNTRY    1511181 non-null  object 
 2   DATE       1511181 non-null  object 
 3   ID_SENDER  1511181 non-null  float64
 4   ID_BRANCH  1511181 non-null  object 
 5   UUID       1489148 non-null  object 
 6   AMOUNT     1511181 non-null  float64
dtypes: float64(2), object(5)
memory usage: 92.2+ MB


In [20]:
df['ID_SENDER'] = df['ID_SENDER'].astype(int).astype(str)

/tmp/ipykernel_35/3582399402.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ID_SENDER'] = df['ID_SENDER'].astype(int).astype(str)


In [22]:
df['DATE'] = pd.to_datetime(df['DATE'])

/tmp/ipykernel_35/2541906660.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DATE'] = pd.to_datetime(df['DATE'])


In [28]:
df['PAYER_COUNTRY'] = df['PAYER'] +'_'+df['COUNTRY']

/tmp/ipykernel_35/4234537024.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['PAYER_COUNTRY'] = df['PAYER'] +'_'+df['COUNTRY']


In [33]:
df['SENDER_BRANCH'] = df['ID_SENDER']+'_'+df['ID_BRANCH']

/tmp/ipykernel_35/4019911665.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['SENDER_BRANCH'] = df['ID_SENDER']+'_'+df['ID_BRANCH']


In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1511181 entries, 0 to 1566739
Data columns (total 9 columns):
 #   Column         Non-Null Count    Dtype         
---  ------         --------------    -----         
 0   PAYER          1511181 non-null  object        
 1   COUNTRY        1511181 non-null  object        
 2   DATE           1511181 non-null  datetime64[ns]
 3   ID_SENDER      1511181 non-null  object        
 4   ID_BRANCH      1511181 non-null  object        
 5   UUID           1489148 non-null  object        
 6   AMOUNT         1511181 non-null  float64       
 7   PAYER_COUNTRY  1511181 non-null  object        
 8   SENDER_BRANCH  1511181 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(7)
memory usage: 115.3+ MB


In [35]:
df.nunique()

PAYER                87
COUNTRY              46
DATE                 29
ID_SENDER         39045
ID_BRANCH          4293
UUID             826244
AMOUNT            20720
PAYER_COUNTRY       122
SENDER_BRANCH    888920
dtype: int64

In [36]:
df[df.UUID.isnull()]

,PAYER,COUNTRY,DATE,ID_SENDER,ID_BRANCH,UUID,AMOUNT,PAYER_COUNTRY,SENDER_BRANCH
291,BANCO DE ORO (BDO),PHILIPPINES,2022-06-07,169,A16167,None,1500.74,BANCO DE ORO (BDO)_PHILIPPINES,169_A16167
350,ENVIOS CON CLABE (APPRIZA),MEXICO,2022-06-07,799,A14227,None,83.00,ENVIOS CON CLABE (APPRIZA)_MEXICO,799_A14227
380,INTERMEX,MEXICO,2022-06-07,56,A22562,None,250.00,INTERMEX_MEXICO,56_A22562
444,BANCOPPEL (APPRIZA),MEXICO,2022-06-07,479,A00922,None,390.00,BANCOPPEL (APPRIZA)_MEXICO,479_A00922
613,BANRURAL (RYT),GUATEMALA,2022-06-25,36,A22562,None,600.00,BANRURAL (RYT)_GUATEMALA,36_A22562
...,...,...,...,...,...,...,...,...,...
1566473,ELEKTRA (MEXICO),MEXICO,2022-06-11,1143,A18124,None,354.00,ELEKTRA (MEXICO)_MEXICO,1143_A18124
1566476,BANCO INDUSTRIAL,GUATEMALA,2022-06-11,477,A19052,None,990.00,BANCO INDUSTRIAL_GUATEMALA,477_A19052
1566489,BANORTE (UT),MEXICO,2022-06-11,1756,A18124,None,1700.00,BANORTE (UT)_MEXICO,1756_A18124
1566592,BANCO INDUSTRIAL,GUATEMALA,2022-06-11,429,A17633,None,2000.00,BANCO INDUSTRIAL_GUATEMALA,429_A17633


In [38]:
def fetch_data(start_date, end_date):
#    conn = connect(s3_staging_dir='s3://viamericas-datalake-dev-us-east-1-283731589572-athena/queryresults/',
#                   region_name='us-east-1')

    query = f'''
    SELECT
        CAST(RTRIM(p.NAME_MAIN_BRANCH) AS VARCHAR(60)) AS PAYER,
        RTRIM(co.NAME_COUNTRY) AS COUNTRY,
        CAST(DATE_FORMAT(a.DATE_RECEIVER, '%Y-%m-%d') AS DATE) AS DATE,
        a.ID_SENDER,
        a.ID_BRANCH,
        s.UUID,
        SUM(CASE WHEN (a.ID_FLAG_RECEIVER = 'A' OR a.ID_FLAG_RECEIVER = 'C') THEN 0 ELSE a.NET_AMOUNT_RECEIVER END) AS AMOUNT
    FROM
        AwsDataCatalog.viamericas.RECEIVER a
        INNER JOIN AwsDataCatalog.viamericas.GROUP_BRANCH p ON p.ID_MAIN_BRANCH = CASE WHEN a.ID_MAIN_BRANCH_EXPIRED IS NULL THEN RTRIM(a.ID_MAIN_BRANCH_SENT) ELSE RTRIM(a.ID_MAIN_BRANCH_EXPIRED) END
        INNER JOIN AwsDataCatalog.viamericas.COUNTRY co ON a.ID_COUNTRY_RECEIVER = co.ID_COUNTRY
        LEFT JOIN AwsDataCatalog.viamericas.BRANCH b ON a.ID_BRANCH = b.ID_BRANCH
        LEFT JOIN AwsDataCatalog.viamericas.SENDER s ON a.ID_SENDER = s.id_sender AND a.ID_BRANCH = s.ID_BRANCH -- Union con la tabla SENDER
    WHERE
        NOT (a.ID_MAIN_BRANCH_SENT LIKE 'M%')
        AND NOT (a.ID_BRANCH LIKE 'T%')
        AND SUBSTRING(b.id_branch, 1, 1) IN (SELECT SUBSTRING(PREFIX, 1, 1) FROM AwsDataCatalog.viamericas.BRANCH_PREFIX P WHERE TRIM(b.ID_COUNTRY) = TRIM(P.ID_COUNTRY))
        AND a.DATE_RECEIVER >= TIMESTAMP '{start_date} 00:00:00.000'
        AND a.DATE_RECEIVER < TIMESTAMP '{end_date} 00:00:00.000'
        AND b.ID_LOCATION IS NOT NULL
        AND b.ID_LOCATION NOT IN ('MD0010', 'MD0952', 'AK0003', 'CA3897', 'NY1130', 'MD0696', 'FL1933', 'AK0004', 'CA4046', 'NY1221', 'MD0623', 'MD1003', 'MD1018', 'AK0008', 'CA4291', 'NY1346', 'CA4350', 'NY1381','OK0236','FL2287','CA4391','AK0009', 'NY1397','CA4392','AK0010','FL2288','NY1399','AK0012','CA4396','NY1402','FL2289','AK0013', 'CA4418','NY1410','FL2301','AK0014', 'CA4428', 'NY1413')
    GROUP BY
        RTRIM(p.NAME_MAIN_BRANCH),
        RTRIM(co.NAME_COUNTRY),
        CAST(DATE_FORMAT(a.DATE_RECEIVER, '%Y-%m-%d') AS DATE),
        a.ID_SENDER,
        a.ID_BRANCH,
        s.UUID
    '''
    
    df = pd.read_sql(query, conn)
    df = df[df['AMOUNT'] != 0]  # Filtrar registros con AMOUNT distinto de cero
    df['ID_SENDER'] = df['ID_SENDER'].astype(int).astype(str)
    df['DATE'] = pd.to_datetime(df['DATE'])
    df['PAYER_COUNTRY'] = df['PAYER'] + '_' + df['COUNTRY']
    df['SENDER_BRANCH'] = df['ID_SENDER'] + '_' + df['ID_BRANCH']
    
    return df

In [39]:
# Llamada a la función con las fechas deseadas
start_date = '2021-12-01'
end_date = '2021-12-31'
data_Dec21 = fetch_data(start_date, end_date)

/tmp/ipykernel_35/1981944711.py:37: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [40]:
data_Dec21

,PAYER,COUNTRY,DATE,ID_SENDER,ID_BRANCH,UUID,AMOUNT,PAYER_COUNTRY,SENDER_BRANCH
0,ELEKTRA (MEXICO),MEXICO,2021-12-28,9003,A14272,b90a0a08-4e12-4c06-afc6-fed53d7b4cc7,1300.0,ELEKTRA (MEXICO)_MEXICO,9003_A14272
1,G Y T CONTINENTAL,GUATEMALA,2021-12-28,2037,A09694,aff45789-b982-48b0-962a-f01c14711536,400.0,G Y T CONTINENTAL_GUATEMALA,2037_A09694
2,BANCOPPEL (APPRIZA),MEXICO,2021-12-28,4201,A19216,ecb7ff29-1cb7-47ad-b7a0-1a216e0ceb24,1000.0,BANCOPPEL (APPRIZA)_MEXICO,4201_A19216
3,CARIBE EXPRESS,DOMINICAN REPUBLIC,2021-12-28,3527,A19216,ee676498-db63-4434-a940-e92f5d74e12f,1000.0,CARIBE EXPRESS_DOMINICAN REPUBLIC,3527_A19216
4,BANRURAL (RYT),GUATEMALA,2021-12-28,402,A19771,2a8aa419-19c1-46ae-946a-7ce46c6b21d5,300.0,BANRURAL (RYT)_GUATEMALA,402_A19771
...,...,...,...,...,...,...,...,...,...
1391958,ELEKTRA (BTS),GUATEMALA,2021-12-23,13026,A19034,ee61df1d-bd43-489a-8270-1e694ee24910,165.0,ELEKTRA (BTS)_GUATEMALA,13026_A19034
1391959,BANCO INDUSTRIAL,GUATEMALA,2021-12-23,11558,A19034,93bcb5f6-7deb-4684-af8e-6d42fd96e5e4,600.0,BANCO INDUSTRIAL_GUATEMALA,11558_A19034
1391960,BANCO INDUSTRIAL,GUATEMALA,2021-12-23,7623,A19034,ab7cf4bd-73ea-4dbb-b612-150f6f55bb34,259.0,BANCO INDUSTRIAL_GUATEMALA,7623_A19034
1391961,BANCO INDUSTRIAL,GUATEMALA,2021-12-23,5995,A19034,5566547e-ee6f-4318-8b07-8035a5cd3580,309.0,BANCO INDUSTRIAL_GUATEMALA,5995_A19034


In [41]:
data_Dec21.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1325729 entries, 0 to 1391962
Data columns (total 9 columns):
 #   Column         Non-Null Count    Dtype         
---  ------         --------------    -----         
 0   PAYER          1325729 non-null  object        
 1   COUNTRY        1325729 non-null  object        
 2   DATE           1325729 non-null  datetime64[ns]
 3   ID_SENDER      1325729 non-null  object        
 4   ID_BRANCH      1325729 non-null  object        
 5   UUID           1303044 non-null  object        
 6   AMOUNT         1325729 non-null  float64       
 7   PAYER_COUNTRY  1325729 non-null  object        
 8   SENDER_BRANCH  1325729 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(7)
memory usage: 101.1+ MB


In [42]:
data_Dec21.nunique()

PAYER                86
COUNTRY              35
DATE                 30
ID_SENDER         31565
ID_BRANCH          4033
UUID             724469
AMOUNT            17404
PAYER_COUNTRY       106
SENDER_BRANCH    791012
dtype: int64

In [44]:
# Llamada a la función con las fechas deseadas
start_date = '2021-01-01'
end_date = '2021-01-31'
data_Jan21 = fetch_data(start_date, end_date)

/tmp/ipykernel_35/1981944711.py:37: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [45]:
data_Jan21

,PAYER,COUNTRY,DATE,ID_SENDER,ID_BRANCH,UUID,AMOUNT,PAYER_COUNTRY,SENDER_BRANCH
0,BANRURAL (RYT),GUATEMALA,2021-01-01,295,A15410,1e459032-a0e4-4601-bdd9-d9b999b40ec1,100.0,BANRURAL (RYT)_GUATEMALA,295_A15410
1,BANCO AGRICOLA,EL SALVADOR,2021-01-01,708,A15410,60e4b878-1b0f-42e8-9898-41bf82b926ab,190.0,BANCO AGRICOLA_EL SALVADOR,708_A15410
2,BANCO AGRICOLA,EL SALVADOR,2021-01-01,104,A15410,0f66da83-2f93-4320-91a2-f1f9ef8634a6,120.0,BANCO AGRICOLA_EL SALVADOR,104_A15410
3,BANCO AGRICOLA,EL SALVADOR,2021-01-01,185,A15410,896bf1c8-e4d9-498a-85a5-12a7f72294fc,420.0,BANCO AGRICOLA_EL SALVADOR,185_A15410
4,WALMART (UT),MEXICO,2021-01-01,387,A17533,5463eda1-34bc-4414-8b61-a1463ef8f858,315.0,WALMART (UT)_MEXICO,387_A17533
...,...,...,...,...,...,...,...,...,...
911034,CAJA POPULAR MEXICANA (UT),MEXICO,2021-01-29,4270,A17395,4839879a-c804-4221-a43a-3e8149a6eb86,130.0,CAJA POPULAR MEXICANA (UT)_MEXICO,4270_A17395
911035,ELEKTRA (MEXICO),MEXICO,2021-01-29,426,A17395,6835e62f-ffd6-48b9-9d54-c69dbc681a29,500.0,ELEKTRA (MEXICO)_MEXICO,426_A17395
911036,G Y T CONTINENTAL,GUATEMALA,2021-01-29,1024,A17523,9a7f2d8d-85b3-4541-b0a5-9f2b9618356e,194.0,G Y T CONTINENTAL_GUATEMALA,1024_A17523
911037,BANORTE (UT),MEXICO,2021-01-29,3557,A17395,2ec72842-67d4-4daf-8ecb-1117e40a4ecf,200.0,BANORTE (UT)_MEXICO,3557_A17395
